# tuesday 2023-07-11 action items:
* ~~Debug Beirut, see where the source of decreased knn weight~~
* ~~re run an accessibility simulation, as well as a betweenness flow simulation for beirut~~
* ~~Once simulation accessibility results done, send to danie~~ addd flows to folder and send another email
* ~~create visuak maos for scenarios showing increase in reach.. Add to google slides~~
* ~~Homes to > transit - parks - amenities - schools~~
* ~~geometric - no turns, no elastic weight, no decay ....~~
* ~~la_extended16_manualFixed~~
* ~~Run Somerville~~
* ~~RUn LA~~
* ~~look at Andres' email about brooklyn , run a simulation using the old code in the workstation~~
* ~~look at Lui's email, see if I need to do anything about it~~
* ~~create OD pairing table for LA, send to Andres~~
* ~~RUN A FKOW MODEL FOR HOMES TO MOSKS, USING THE SAME PARAMETERS, WITH KNN , FOR ALL SCENARIOS, SEND TO DANIEL~~
* ~~CLip lui's dataset into the area that daniel chose, share with andres~~
*~~ FOR the clip, use this file "C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\02_Edited\NYC_Sidewalk_Edit_July\BK_clipping.geojson"~~
* ~~make sure the new node insertion code works and matches well~~

## effecient node edge creation
* ~~Make sure the functionality to construct nodes-edges works for a safey buffer~~
* ~~Isolate the node insertion code into script~~
* ~~commit script to my branch~~
* ~~email orion about it, mention how easy to integrate.~~
* ~~look into issue with redundant edges~~

## Testing
* ~~structure tests so its a one csv: first n rows are settings, next m rows are edges flows, with first column being same segment ids from network file~~
* ~~standard test case is four files: network.jeojson - origins.geojson - destination.jeojson - testflows.csv~~
* ~~function reads a csv into list of dicts: test settings and series of output.~~
* ~~build Harvard Square testflows.csv~~
~~* make sure this config runs on Harvard Square~~



# TODO
## Functionality List 1
* add visualization functionality to new version of Madina so testing is easier
* in network, add  connector segments as a separate layer (Maybe not very needed if origins and destinations visually are snapoped...)
* Commit visualization updates to github


## Testing
* digitize three test cases for manhattan, visualize OD snapping, show where differences (if any) come from between Rhino  and Python in the NYC case.
* Put maps into slides
* CHeck somerville, to see if we delete the same number of edges, with and without the effecient code "C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\Cities\Somerville\Data\Somerville_network_1000m_buffer.geojson"

## Functionality List 2
* build the betweenness simulation workflow, combine with accissibility
* include Logger as a class, and zonal would have an instance of that class. Logger handles event documentation, and captures output.
* for each origin, have an origin record, showing its knn weight, reach, gravity towards each of its destination, header rows showing settings and parameters
* for the network, columns of od flows, headed  by settings and parameters rows that explicitly detail units, weight type, calibration status, .. any relevant data, settings and parameters..
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination, Document all relevant settings
* show how this generalizes to the pairings.csv, iterating over moodel settings, iterating over scenarios
* Make a finalized github commit including testing script, betweenness flow notebook. 
* Deploy to workstation.


## Functionality List 3
* given the simulation results, and the censor locations, find a way to calibrate counts and generate beta coeffecients (Beirut, SOmerville, NYC)
* enable Jupyter notebook server access from workstation.
* Figure out a way to do the "Cities" dropbox folder, people can add data through dropbox, run simulations through Jupyter Server, get results through Dropbox




## Running Cases
* Send output of LA to Nico when done
* send Brooklyn output when done
* be prepared to run entire NYC simulation, be mindful that daniel already sent more network parts


## Map Cleaning
https://www.dropbox.com/scl/fo/d6ugcyt3aanfb3yk4n9va/h?dl=0&rlkey=1c2giilvzoyygcxrlry7fscr9



! pip install tqdm
#from tqdm import tqdm
import sys
print(sys.version)

In [8]:
import sys
sys.path.append('../')



from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight

import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np

from geopandas import GeoDataFrame, GeoSeries


print (f"{'test name':30s} | {'madina_flow_sum':15s} | {'Rhino_flow_sum':15s} | {'sum_diff':8s} | {'sum_smlr_%':10s} | {'sle_mean':8s} | {'sle_median':10s} | {'sle_max':8s} | {'sle>0.1%':8s} | {'sle>1.0%':8s} | {'sle>3.0%':8s} | {'sle>5.0%':8s}")
#for test_case in os.listdir("Test Cases"):
for test_case in ['Harvard Square', 'Somerville']:   #
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    if test_case == 'Somerville':
        resdy_tests = test_config.index[0:4]
    else:
        ready_tests = test_config.index

    for test_idx in ready_tests:

        harvard_square = Zonal()

        harvard_square.load_layer(
            layer_name='streets',
            file_path=  test_case_folder + test_config.at[test_idx, 'Network_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Origin_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Origin_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Destination_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Destination_File']
            )
        

        harvard_square.create_street_network(
            source_layer='streets', 
            discard_redundant_edges=False,
            node_snapping_tolerance=0.01,  # TODO: check for sensitivity... pick one as default snapping.
            weight_attribute=test_config.at[test_idx, 'Network_Cost'] if test_config.at[test_idx, 'Network_Cost'] != "Geometric" else None
        )

        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")
        harvard_square.network.nodes,  harvard_square.network.edges = _discard_redundant_edges(harvard_square.network.nodes, harvard_square.network.edges)
        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")

        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Origin_Name'], 
            label='origin', 
            weight_attribute=test_config.at[test_idx, 'Origin_Weight'] if test_config.at[test_idx, 'Origin_Weight'] != "Count" else None
        )
        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Destination_Name'], 
            label='destination', 
            weight_attribute=test_config.at[test_idx, 'Destination_Weight'] if test_config.at[test_idx, 'Destination_Weight'] != "Count" else None
        )

        harvard_square.create_graph()

        node_gdf = harvard_square.network.nodes
        origin_gdf = node_gdf[node_gdf['type'] == 'origin']

        harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]

        harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn penalty']
        harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn threshold']

        if test_config.at[test_idx, 'Elastic_weights']:
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, 'Beta'],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']


        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, 'Beta'],
            path_detour_penalty='equal', # "equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=6,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'Flow_Name']].sum()


        ## create segment level comparison
        # creating connector lines

        import shapely.geometry as geo
        simulated_betweenness = return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).drop_duplicates(subset=['parent_street_id']).set_index("parent_street_id")
        simulated_betweenness = harvard_square.layers["streets"].gdf[["geometry", "__GUID"]].join(simulated_betweenness).set_index("__GUID")

        test_name = test_config.at[test_idx, 'Flow_Name']
        test_betweenness = test_flows[['__GUID', test_name]].set_index("__GUID").rename(columns = {test_name: "test_flow"})


        comparison = simulated_betweenness.join(test_betweenness)
        comparison["difference"] = comparison["simulated_betweenness"] - comparison["test_flow"]
        comparison["difference_pct"] = abs(comparison["difference"]) / comparison["simulated_betweenness"] *100
        # segment level error
        sle = comparison[~comparison["difference_pct"].isna()]['difference_pct']

        #'''
        node_gdf = harvard_square.network.nodes
        edge_gdf = harvard_square.network.edges


        origin_layer = harvard_square.layers[test_config.at[test_idx, 'Origin_Name']].gdf
        origin_nodes = node_gdf[node_gdf['type'] == 'origin']
        origin_joined = origin_layer.join(origin_nodes.set_index('source_id'),lsuffix='_origin')
        origin_joined['connector_line'] = origin_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_origin"]]), axis=1)
        origin_joined["geometry"] = origin_joined['connector_line']


        destination_layer = harvard_square.layers[test_config.at[test_idx, 'Destination_Name']].gdf
        destination_nodes = node_gdf[node_gdf['type'] == 'destination']
        destination_joined = destination_layer.join(destination_nodes.set_index('source_id'),lsuffix='_destination')
        destination_joined['connector_line'] = destination_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_destination"]]), axis=1)
        destination_joined["geometry"] = destination_joined['connector_line']

        streets = harvard_square.layers["streets"].gdf 
        network_file = gpd.read_file(test_case_folder + test_config.at[test_idx, 'Network_File'], engine='pyogrio')


        flow_difference = comparison[
            ((comparison['test_flow' ] > 0) & (comparison['simulated_betweenness'] == 0)) | 
            ((comparison['test_flow' ] == 0) & (comparison['simulated_betweenness'] > 0))
        ]
        harvard_square.create_map(
            [
                #{'gdf': streets[streets[test_config.at[test_idx, 'Network_Cost']] > 0], 'color': [255, 255, 255], 'text': test_config.at[test_idx, 'Network_Cost']},
                {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
                {'gdf': edge_gdf[edge_gdf['betweenness'] > 0], 'color': ['125, 125, 0'], 'text': 'betweenness', 'opacity': 0.2},
                #{'gdf': comparison[abs(comparison["difference"]) > 0.01], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference'},
                {'gdf': comparison[(comparison["difference_pct"] >= 0.1) & (comparison["difference_pct"] < 100)], 'color_by_attribute': 'difference_pct', 'color_method': 'gradient', 'text': 'difference_pct'},
                {'gdf': edge_gdf[edge_gdf['snapped'] == True], 'color': [255, 0, 255], 'opacity': 0.2},
                {'gdf': network_file[network_file["geometry"].geom_type == 'Polygon'], 'color': [125, 0, 125], 'text': '__GUID'},
                {'gdf': flow_difference, 'color': [255, 255, 0] , 'text': 'difference'},        
                #{'gdf': comparison[comparison['difference'] != 0], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference', 'opacity':  0.1},
                {'gdf': origin_layer, 'color': [0, 0, 255]},
                {'gdf': origin_joined[['geometry']], 'color': [0, 0, 255]},
                {'gdf': destination_layer, 'color': [255, 0, 0]},
                {'gdf': destination_joined[['geometry']], 'color': [255, 0, 0]},
                #{'gdf': harvard_square.network.nodes[['geometry', 'type', 'weight']].reset_index(), 'color': [255, 0, 255], 'text': 'id'},
            ],
            save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
        )
        #'''
        #print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'Flow_Name'][:30]:30s} | {simulated_sum_of_flow:15.2f} | {test_flow:15.2f} | {simulated_sum_of_flow - test_flow:8.2f} | {1-(simulated_sum_of_flow - test_flow)/ test_flow:10.2%} | {sle.mean():7.4f}% | {sle.median():9.4f}% | {sle.max():7.4f}% | {sle[sle > 0.1].count():8} | {sle[sle > 1.0].count():8} | {sle[sle > 3.0].count():8} | {sle[sle > 5.0].count():8}")
        #print ("DOne Case...")
        #break

test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
_Betweenness5                  |        13478.14 |        13394.55 |    83.59 |     99.38% |  0.0288% |    0.0000% |  3.8299% |        1 |        1 | 

KeyError: 'Layer with label Amenities is already in Zonal object'

In [ ]:
With splitting redundant nodes. low tolerance = 0.000001


test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
Somerville_Bus_Subway_Geometri |          709.44 |          713.14 |    -3.69 |    100.52% |  0.5027% |    0.1954% |  3.9604% |      149 |       62 |        3 |        0
Somerville_Bus_Subway          |          740.19 |          743.95 |    -3.76 |    100.51% |  0.5074% |    0.2539% |  3.9604% |      173 |       39 |        3 |        0
Somerville_Homes_Subway        |       372834.66 |       389383.31 | -16548.65 |    104.25% |     inf% |    3.4592% |     inf% |     2562 |     2191 |     1467 |      973
Somerville_Jobs_Subway         |       142623.89 |       149908.30 | -7284.41 |    104.86% |     inf% |    3.3898% |     inf% |     2077 |     1691 |     1237 |      854

In [3]:
.sort_values('difference_pct')

,geometry,simulated_betweenness,test_flow,difference,difference_pct
__GUID,,,,,
9c2a0a1e-93d0-433d-8ef7-28dbd6399747,"LINESTRING (230790.763 905654.025, 230779.586 ...",68.441179,68.513274,-0.072095,0.105338
2716b82e-dd5a-4d56-9600-f1be9ca7dc65,"LINESTRING (235426.555 902244.112, 235340.448 ...",249.194789,249.472320,-0.277532,0.111371
16900608-afb7-4a1f-ad89-2c409b1f787f,"LINESTRING (231614.209 904322.518, 231610.231 ...",2.617736,2.620768,-0.003033,0.115846
5029006c-04ae-468c-9864-57d94d93f9c7,"LINESTRING (230943.967 904147.430, 230959.739 ...",19.276246,19.299348,-0.023102,0.119849
44fe89a6-f0dc-4f6a-9248-c7eb987d307b,"LINESTRING (230958.695 904156.903, 231094.268 ...",19.205963,19.229048,-0.023086,0.120200
...,...,...,...,...,...
62f07ef0-7a9e-449a-9f9f-8fb1bf469642,"LINESTRING (231089.649 904550.175, 231044.921 ...",5.621143,10.415312,-4.794169,85.288160
cb17843e-c50e-4319-a683-9680bd442913,"LINESTRING (235119.769 903581.204, 235111.791 ...",0.038618,0.072567,-0.033949,87.909842
1ddb4033-54ce-48b7-821d-ef7a2a43fe5c,"LINESTRING (235172.561 903621.278, 235157.705 ...",0.115854,0.217701,-0.101847,87.909843


In [ ]:
flow_difference = comparison[
    ((comparison['test_flow' ] > 0) & (comparison['simulated_betweenness'] == 0)) | 
    ((comparison['test_flow' ] == 0) & (comparison['simulated_betweenness'] > 0))
]
harvard_square.create_map(
    [
        #{'gdf': streets[streets[test_config.at[test_idx, 'Network_Cost']] > 0], 'color': [255, 255, 255], 'text': test_config.at[test_idx, 'Network_Cost']},
        {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
        {'gdf': edge_gdf[edge_gdf['betweenness'] > 0], 'color': ['125, 125, 125'], 'text': 'betweenness', 'opacity': 0.1},
        {'gdf': comparison[abs(comparison["difference"]) > 0.01], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference'},
        {'gdf': edge_gdf[edge_gdf['snapped'] == True], 'color': [255, 0, 255], 'text': 'snapping_distance'},
        {'gdf': network_file[network_file["geometry"].geom_type == 'Polygon'], 'color': [125, 0, 125], 'text': '__GUID'},
        {'gdf': flow_difference, 'color': [255, 255, 0] , 'text': 'difference'},        
        #{'gdf': comparison[comparison['difference'] != 0], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference', 'opacity':  0.1},
        {'gdf': origin_layer, 'color': [0, 0, 255]},
        {'gdf': origin_joined[['geometry']], 'color': [0, 0, 255]},
        {'gdf': destination_layer, 'color': [255, 0, 0]},
        {'gdf': destination_joined[['geometry']], 'color': [255, 0, 0]},
        #{'gdf': harvard_square.network.nodes[['geometry', 'type', 'weight']].reset_index(), 'color': [255, 0, 255], 'text': 'id'},
    ],
    save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
)

In [ ]:
harvard_square.layers.layers

In [ ]:
harvard_square.network.edges

In [ ]:
harvard_square.network.edges.set_index("parent_street_id").loc[5759]

In [ ]:
import geopandas as gpd


In [ ]:
network_file[network_file["geometry"].geom_type == 'Polygon']

In [ ]:
network_file[(network_file["PercLen"] < 10) & (network_file["PercLen"] > 0)]["PercLen"]
network_file[network_file["__Length"] < 0.1]

network_file[network_file["PercLen"] > network_file["geometry"].length]
network_file[network_file["PercLen"] > 0][['__Length', 'PercLen']]


In [ ]:
network_file[network_file['PercLen'] != 0]
network_file['weight'] = network_file.apply(lambda x: x["PercLen"] if x["PercLen"] != 0 else x["geometry"].length, axis=1)
network_file[network_file['PercLen'] == 0][['PercLen', 'weight']]
weight_series = network_file.apply(lambda x: x["PercLen"] if x["PercLen"] != 0 else x["geometry"].length, axis=1)
weight_series

In [ ]:

harvard_square.layers['streets'].gdf['PercLen'].hist()

In [ ]:
edge_gdf = harvard_square.network.edges
edge_gdf[edge_gdf['weight'] <= 0.1]

In [ ]:
origin_nodes
destination_nodes
test_config.at[test_idx, 'Elastic_weights']
origin_nodes
return_dict['edge_gdf']["betweenness"].max()
type(test_config.at[test_idx, 'Elastic_weights'])

test_config.at[test_idx, 'Elastic_weights']

In [ ]:
simulated_betweenness['simulated_betweenness'].sum()
simulated_betweenness.index.unique().shape

In [ ]:
comparison[comparison["difference"] < 0]
comparison["difference"]
test_betweenness.sum()
test_betweenness.index.unique().shape

In [ ]:
comparison[comparison["difference_pct"] > 1]

In [ ]:
return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).set_index("parent_street_id")

In [ ]:
#how{‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}
test_betweenness.join(simulated_betweenness, how='right')


In [ ]:
simulated_betweenness = return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).drop_duplicates(subset=['parent_street_id']).set_index("parent_street_id")
simulated_betweenness = harvard_square.layers["streets"].gdf[["geometry", "__GUID"]].join(simulated_betweenness).set_index("__GUID")

test_name = test_config.at[test_idx, 'test_name']
test_betweenness = test_flows[['__GUID', test_name]].set_index("__GUID").rename(columns = {test_name: "test_flow"})


#comparison = simulated_betweenness.join(test_betweenness)
comparison = test_betweenness.join(simulated_betweenness)
comparison["difference"] = comparison["simulated_betweenness"] - comparison["test_flow"]
comparison["difference_pct"] = abs(comparison["difference"]) / comparison["simulated_betweenness"] *100
mean_error = comparison[~comparison["difference_pct"].isna()]['difference_pct'].mean() 

list(comparison[~comparison["difference_pct"].isna()]['difference_pct'])
comparison

In [ ]:
test_betweenness

In [ ]:
return_dict['edge_gdf'][['betweenness', 'parent_street_id']]

# COnstructing the Manhattan Case..
## Don't forget to check for node insertion, set  return_all=False in the spatial index

In [ ]:
import time
start = time.time()

buildings_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Home_PT_6538.geojson"
subway_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Metro_PT_6538.geojson"
network_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\network_clipped_dupremovedAS.geojson"

import sys
sys.path.append('../')
from madina.zonal.zonal import Zonal


harvard_square = Zonal()

print(f"{(time.time()-start)*1000:6.2f}ms\t imports done, object created")
start = time.time()

#harvard_square.load_layer(
#    layer_name='streets',
#    file_path=network_file
#    )


print(f"{(time.time()-start)*1000:6.2f}ms\t street data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='buildings',
    file_path=buildings_file
    )

buildings_gdf = harvard_square.layers['buildings'].gdf
harvard_square.layers['buildings'].gdf = buildings_gdf[~buildings_gdf['FID'].isin([27967, 9140, 3974])]

print(f"{(time.time()-start)*1000:6.2f}ms\t building data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='subway',
    file_path=subway_file
    )

print(f"{(time.time()-start)*1000:6.2f}ms\t subway data loaded")

In [ ]:
start = time.time()

harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False, 
    node_snapping_tolerance=0
)

print(f"{(time.time()-start)*1000:6.2f}ms\t street network created")
start = time.time()

harvard_square.insert_node(
    layer_name='buildings', 
    label='origin', 
    weight_attribute='TotalPop'
)


print(f"{(time.time()-start)*1000:6.2f}ms\t origins insertes")
start = time.time()

harvard_square.insert_node(
    layer_name='subway', 
    label='destination', 
    weight_attribute='line_ent_st'
)

print(f"{(time.time()-start)*1000:6.2f}ms\t destinations insertes")


start = time.time()

harvard_square.create_graph(light_graph=True, d_graph=True)

print(f"{(time.time()-start)*1000:6.2f}ms\t graph created insertes")
start = time.time()

return_dict = parallel_betweenness(
    harvard_square.network,
    search_radius=800,
    detour_ratio=1.15,
    decay=False,
    decay_method='exponent',  # "power", "exponent"
    beta=0.004,
    path_detour_penalty="equal",  # "power", "exponent", "equal"
    origin_weights=True,
    closest_destination=False,
    destination_weights=True, 
    # perceived_distance=False,
    num_cores=8,
    light_graph=True,
    turn_penalty=False,
)
simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()

print(f"{(time.time()-start)*1000:6.2f}ms\t Betweenness estimated")
start = time.time()

In [ ]:
joined_results = harvard_square.layers['streets'].gdf.join(harvard_square.network.edges[['parent_street_id', 'betweenness']].set_index('parent_street_id'))
joined_results[['__GUID', 'betweenness', 'geometry']].to_csv('2023-07-07 manhattan betweenness flow test.csv')
joined_results[['__GUID', 'betweenness', 'geometry']].to_file('2023-07-07 manhattan betweenness flow test.geoJSON', driver="GeoJSON")

In [10]:
harvard_square.create_map(
    [
        #{
            #'layer': 'streets',
            #'color': [125, 125, 125],
        #},
        {
            'gdf': harvard_square.layers['homes'].gdf,
            'color_by_attribute': 'line_ent_st',
            "color_method": 'gradient'
        }
    ],
    save_as="map.html"
)

KeyError: 'No layer in the Zonal object has the label homes'

## Need to instal pydeck jupyter nb extension

`jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck`

`jupyter nbextension enable --sys-prefix --py pydeck`
